In [1]:
#TO DO save intermediate protagonist map to csv
#check gpt-4.1-mini performance via uva api

import os
import requests
import time
import pandas as pd
import openai
client = openai.OpenAI()

kaggle_data = pd.read_csv("data/kaggle_data.csv")

In [2]:
kaggle_data.columns

Index(['movie_name', 'imdb_id', 'character_name_kaggle', 'line_count', 'title',
       'director', 'genres', 'imdb_rating', 'year', 'plot', 'script_filename',
       'script_path', 'script_exists', 'Drama', 'Thriller', 'Comedy', 'Action',
       'Crime', 'Romance', 'Adventure', 'Sci-Fi', 'Mystery', 'Horror',
       'Fantasy', 'Biography', 'imdb_rating_decimal'],
      dtype='object')

In [3]:
if os.path.exists("data/intermediate_protagonist.csv"):
    protagonist_map = pd.read_csv("data/intermediate_protagonist.csv", index_col=0).to_dict()["0"]
    protagonist_map = {str(k).zfill(7): v for k, v in protagonist_map.items()}

else:
    protagonist_map = {}

In [ ]:
#cost before run was 46.96
from openai import OpenAI

# your_api_key = os.getenv("UVA_OPENAI_API_KEY")
# the_base_url_to_always_use = "https://ai-research-proxy.azurewebsites.net/"
# client = OpenAI(api_key=your_api_key, base_url=the_base_url_to_always_use)
client = OpenAI()

for i in range(10, 21):
    imdb_id = kaggle_data.iloc[i]["imdb_id"]
    if str(imdb_id).zfill(7) in protagonist_map:
        continue
    title = kaggle_data.iloc[i]["title"]
    description = kaggle_data.iloc[i]["plot"]
    director = kaggle_data.iloc[i]["director"]

    prompt = f"""Search the internet for the protagonist of the following movie.
    Title: {title}
    Description: {description}
    Director: {director}
    Never write anything other than the name of the protagonist.
    No explanations, no full sentences, no references--just the name of the protagonist.
    If there is no clear protagonist, return an empty string.
    Protagonist name:"""

    response = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "web_search"}],
    input = prompt,
    temperature=0)
    answer = response.output_text

    #write raw output to protagonist map
    protagonist_map[str(imdb_id).zfill(7)] = answer
    if i % 5 == 0:
        print(i)
        pd.Series(protagonist_map).to_csv(f"data/intermediate_protagonists.csv")

Movie: 16 Blocks
Protagonist: Jack Mosley 
Movie: 21 Grams
Protagonist: Paul Rivers 
Movie: 28 Days Later...
Protagonist: Jim
